# Análise dos dados de tumor no cerebro

---



Pacotes a serem utilizados

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import tensorflow as tf
import cv2
import cv2 as cv
import glob
import skimage.io
import shutil
import zipfile
import io
import os
from os import listdir
from PIL import Image, ImageFilter
import imutils
import time
import scipy.ndimage
import copy
from google.colab import files
from random import shuffle
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input, ZeroPadding2D, BatchNormalization, Activation
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import losses
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import morphology
from skimage import measure,filters
from skimage.transform import resize
from skimage.viewer import ImageViewer
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, black_tophat, skeletonize, convex_hull_image, disk
from skimage.data import data_dir
from skimage.util import img_as_ubyte
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from PIL import Image, ImageDraw, ImageFilter

carregando a base de dados

In [ ]:
from google.colab import files
uploaded2 = files.upload()

Saving dados.zip to dados.zip


In [ ]:
data = zipfile.ZipFile('/content/dados.zip', 'r')
data.extractall()

In [ ]:
dir_list = ['/content/Pre_process/No/','/content/Pre_process/Yes/']

In [ ]:
def load_data(dir_list):

    # load all images in a directory
    X = []
    y = []

    for directory in dir_list:
        for filename in listdir(directory):
            image = cv2.imread(directory + filename)
            image = image / 255.
            X.append(image)
            if directory == '/content/Pre_process/Yes/':
                y.append([1])
            else:
                y.append([0])

    X = np.array(X)
    y = np.array(y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')

    return X, y

In [ ]:
X, y = load_data(dir_list)

Number of examples is: 253
X shape is: (253, 256, 256, 3)
y shape is: (253, 1)


divisão treinamento teste e validação

In [ ]:
def split_data(X, y, test_size=0.2):

    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.2)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_val = tf.keras.utils.to_categorical(y_val)

# Modelo

In [ ]:
vgg16 = tf.keras.applications.VGG16(input_shape=(256,256,3), include_top = False, weights= 'imagenet')

In [ ]:
x = vgg16.output
x = Flatten()(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model=Model(inputs=vgg16.input,outputs=out)
#
tf_model.compile(optimizer=tf.keras.optimizers.Adadelta(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
#tf_model.summary()
for layer in tf_model.layers[:20]:
    layer.trainable=False

In [ ]:
history = tf_model.fit(X_train, y_train, batch_size =32, epochs = 150, initial_epoch = 0, validation_data = (X_val, y_val),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=15))

Epoch 1/150
7/7 [==============================] - 8s 1s/step - loss: 0.8577 - accuracy: 0.4558 - val_loss: 0.7187 - val_accuracy: 0.5789
Epoch 2/150
7/7 [==============================] - 7s 1s/step - loss: 0.7963 - accuracy: 0.4465 - val_loss: 0.7215 - val_accuracy: 0.3684
Epoch 3/150
7/7 [==============================] - 7s 1s/step - loss: 0.7833 - accuracy: 0.4977 - val_loss: 0.7292 - val_accuracy: 0.4211
Epoch 4/150
7/7 [==============================] - 7s 1s/step - loss: 0.7567 - accuracy: 0.5209 - val_loss: 0.7399 - val_accuracy: 0.4211
Epoch 5/150
7/7 [==============================] - 7s 1s/step - loss: 0.7531 - accuracy: 0.5442 - val_loss: 0.7415 - val_accuracy: 0.4211
Epoch 6/150
7/7 [==============================] - 7s 1s/step - loss: 0.7193 - accuracy: 0.5628 - val_loss: 0.7439 - val_accuracy: 0.4211
Epoch 7/150
7/7 [==============================] - 7s 1s/step - loss: 0.7330 - accuracy: 0.6093 - val_loss: 0.7434 - val_accuracy: 0.4211
Epoch 8/150
7/7 [=================

In [ ]:
y_predt = tf_model.predict(X_test)
ytest3=np.argmax(y_test,axis=1)
y_predt= np.argmax(y_predt,axis=1)

In [ ]:
resultados[:,0]=y_predt

In [ ]:
cm1 = confusion_matrix(y_predt,ytest3);cm1 #92,101